In [1]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json
import numpy as np

from functions import (get_sector, changes_from_press, get_content_header,
                        get_df, normalise_to_index, save_data)

past_last_date = False


bad_urls = ['https://www.pfizer.com/news/press-release/press-release-detail/imi-sophia-partners-maccabis-research-and-innovation-center']


C:\Users\Aidan\miniconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = []

ticker = 'pfe'

sector = get_sector(ticker)

index_ticker = '^GSPC'
# select by value 

stock_hist = get_df(ticker)
index_hist = get_df(index_ticker)

page_num = 0
# create break
while True:    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"}
    
    s = requests.Session()
    html = requests.get(f'https://www.pfizer.com/news/press-releases?items_per_page=48&page={page_num}', headers=headers)
    

    soup = BeautifulSoup(html.content) 

    articles = soup.find_all('li',attrs={'class':'grid-x'})
    
    if not articles:
        break
    
    for article in articles:
        # remove starting and trailing whitespaces
        date = article.find('helix-core-content', attrs={'slot':'date'}).text.lstrip().rstrip()
        title = article.find('helix-core-heading', attrs={'variant':'h4'}).text.lstrip().rstrip()
        url = article.find('a').get('href')
        url = f'https://www.pfizer.com{url}'
        print(url)
        if url in bad_urls:
            continue
        content = get_content_header(url, 'meta' , {'name':'description'})  
        date = datetime.strptime(date, '%m.%d.%Y').date()
        if date < datetime(2014, 1, 1).date():
            past_last_date = True
            break        
        
        pct_change = changes_from_press(stock_hist, date, 1)
        
        if pct_change != None:
            index_price, norm_price = normalise_to_index(pct_change, date, index_hist)
        else:
            index_price, norm_price = (None, None)

        data.append([str(date), title, url, content, pct_change, index_price, norm_price])
    if past_last_date == True: break
    page_num = page_num + 1
    
    
save_data(data, ticker, sector)

AttributeError: 'NoneType' object has no attribute 'text'

In [7]:
type(ARTICLE)

str